In [12]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [13]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/"

In [14]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [15]:
def file_paths(df):
    
    # Create a column with the JSON name
    #df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]
    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    #mark = ".json"

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [16]:
def string_to_dict(s):
    if ',' not in s and ':' not in s:
        return s
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [17]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = df['variable'] 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

    
    """
    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())

    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]
   
    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')

    """


    def column_paths(df, row_var, get_var, unlist_var, explode_var):

        # Convert to dict if it's a string
        df[row_var] = df[row_var].apply(
            lambda x: string_to_dict(x) if isinstance(x, str) else x)
        
    
        df[get_var] = df.apply(
        lambda row: list(row[row_var].keys()) if isinstance(row[row_var], dict) else row[row_var], axis=1)


        display(df)

        #df[get_var] = df[get_var].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

        

        df = df.explode(get_var)
        
        
        #display(df)

        
        # Extract nested values using keys
        df[unlist_var] = df.apply(lambda row: [row[row_var].get(row[get_var], None)] if isinstance (row[row_var], dict) else None, axis=1)

        
        #display(df[unlist_var])

        # Unlist and explode
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)

        """
        
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)

        #display(df[unlist_var])
        
        df[explode_var] = df[unlist_var]
        df = df.explode(explode_var)
        """

        
    

        return df

    row_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']


    for r, g, u, e in zip(row_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        
    return df


In [18]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)

    return df

In [19]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)
    
    # Strip top-level key
    data = list(data.values())[0]


    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)



    # Delete user specific informations
    #df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)


    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [20]:
input_directory = Path(f'{main_path}TikTok/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/TikTok/Input


In [21]:

for file in input_directory.iterdir():  
    if file.is_file():  
        #print(file)
        structure_donations(file)
        

,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,"[Ad Interests, Instant Form Ads Responses, Off...",,,,
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,"[Block List, Settings]",,,,
2,Comment,"{'Comments': {'App': 'number', 'CommentsList':...",Comment,[Comments],,,,
3,Direct Message,{'Direct Messages': {'ChatHistory': {'Chat His...,Direct Message,[Direct Messages],,,,
4,Income Plus Wallet Transactions,{'Transaction History': {'TransactionsList': '...,Income Plus Wallet Transactions,[Transaction History],,,,
5,Location Review,{'Location Reviews': {'ReviewsList': 'object'}},Location Review,[Location Reviews],,,,
6,Post,"{'Posts': {'VideoList': [{'Date': 'string', 'L...",Post,"[Posts, Recently Deleted Posts]",,,,
7,Profile,"{'AI-Moji': {'CreateDate': 'string', 'AIMojiLi...",Profile,"[AI-Moji, Autofill, Profile Info]",,,,
8,TikTok Shop,{'Communication With Shops': {'CommunicationHi...,TikTok Shop,"[Communication With Shops, Current Payment Inf...",,,,
9,Tiktok Live,"{'Go Live History': {'GoLiveList': 'object'}, ...",Tiktok Live,"[Go Live History, Go Live Settings, Watch Live...",,,,


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,[AdInterestCategories],,,,{'AdInterestCategories': 'string'}
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,[ResponsesList],,,,{'ResponsesList': 'object'}
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,[OffTikTokActivityDataList],,,,{'OffTikTokActivityDataList': 'object'}
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,"[App, BlockList]",,,,"{'App': 'number', 'BlockList': 'object'}"
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Settings,"[App, SettingsMap]",,,,"{'App': 'number', 'SettingsMap': {'Allow DownL..."
2,Comment,"{'Comments': {'App': 'number', 'CommentsList':...",Comment,Comments,"[App, CommentsList]",,,,"{'App': 'number', 'CommentsList': [{'date': 's..."
3,Direct Message,{'Direct Messages': {'ChatHistory': {'Chat His...,Direct Message,Direct Messages,[ChatHistory],,,,{'ChatHistory': {'Chat History with graaciel_:...
4,Income Plus Wallet Transactions,{'Transaction History': {'TransactionsList': '...,Income Plus Wallet Transactions,Transaction History,[TransactionsList],,,,{'TransactionsList': 'object'}
5,Location Review,{'Location Reviews': {'ReviewsList': 'object'}},Location Review,Location Reviews,[ReviewsList],,,,{'ReviewsList': 'object'}
6,Post,"{'Posts': {'VideoList': [{'Date': 'string', 'L...",Post,Posts,[VideoList],,,,"{'VideoList': [{'Date': 'string', 'Link': 'str..."


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values,col_path_2_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,AdInterestCategories,string,,,{'AdInterestCategories': 'string'},string
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,ResponsesList,object,,,{'ResponsesList': 'object'},object
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,object,,,{'OffTikTokActivityDataList': 'object'},object
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,App,number,,,"{'App': 'number', 'BlockList': 'object'}",number
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,BlockList,object,,,"{'App': 'number', 'BlockList': 'object'}",object
...,...,...,...,...,...,...,...,...,...,...
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Purchases,BuyGifts,[BuyGifts],,,"{'SendGifts': {'SendGifts': 'object'}, 'BuyGif...",{'BuyGifts': 'object'}
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Searches,SearchList,"[Date, SearchTerm]",,,"{'SearchList': [{'Date': 'string', 'SearchTerm...","{'Date': 'string', 'SearchTerm': 'string'}"
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Share History,ShareHistoryList,object,,,{'ShareHistoryList': 'object'},object
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,"[Resolution, App Version, IDFA, GAID, Android ...",,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin..."


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values,col_path_2_values,col_path_3_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,AdInterestCategories,string,None,,{'AdInterestCategories': 'string'},string,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,ResponsesList,object,None,,{'ResponsesList': 'object'},object,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,object,None,,{'OffTikTokActivityDataList': 'object'},object,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,App,number,None,,"{'App': 'number', 'BlockList': 'object'}",number,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,BlockList,object,None,,"{'App': 'number', 'BlockList': 'object'}",object,None
...,...,...,...,...,...,...,...,...,...,...,...
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Android ID,string,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,IDFV,string,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Web ID,string,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Watch History,VideoList,Date,string,,"{'VideoList': [{'Date': 'string', 'Link': 'str...","{'Date': 'string', 'Link': 'string'}",string


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values,col_path_2_values,col_path_3_values,col_path_4_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,AdInterestCategories,string,None,None,{'AdInterestCategories': 'string'},string,None,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,ResponsesList,object,None,None,{'ResponsesList': 'object'},object,None,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,object,None,None,{'OffTikTokActivityDataList': 'object'},object,None,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,App,number,None,None,"{'App': 'number', 'BlockList': 'object'}",number,None,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,BlockList,object,None,None,"{'App': 'number', 'BlockList': 'object'}",object,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Android ID,string,None,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string,None
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,IDFV,string,None,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string,None
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Web ID,string,None,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string,None
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Watch History,VideoList,Date,string,None,"{'VideoList': [{'Date': 'string', 'Link': 'str...","{'Date': 'string', 'Link': 'string'}",string,None


...

,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,"[Ad Interests, Instant Form Ads Responses, Off...",,,,
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,"[Block List, Settings]",,,,
2,Comment,"{'Comments': {'App': 'number', 'CommentsList':...",Comment,[Comments],,,,
3,Direct Message,{'Direct Messages': {'ChatHistory': {}}},Direct Message,[Direct Messages],,,,
4,Income Plus Wallet Transactions,{'Transaction History': {'TransactionsList': '...,Income Plus Wallet Transactions,[Transaction History],,,,
5,Location Review,{'Location Reviews': {'ReviewsList': 'object'}},Location Review,[Location Reviews],,,,
6,Post,"{'Posts': {'VideoList': 'object'}, 'Recently D...",Post,"[Posts, Recently Deleted Posts]",,,,
7,Profile,"{'AI-Moji': {'CreateDate': 'string', 'AIMojiLi...",Profile,"[AI-Moji, Autofill, Profile Info]",,,,
8,TikTok Shop,{'Communication With Shops': {'CommunicationHi...,TikTok Shop,"[Communication With Shops, Current Payment Inf...",,,,
9,Tiktok Live,"{'Go Live History': {'GoLiveList': 'object'}, ...",Tiktok Live,"[Go Live History, Go Live Settings, Watch Live...",,,,


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,[AdInterestCategories],,,,{'AdInterestCategories': 'string'}
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,[ResponsesList],,,,{'ResponsesList': 'object'}
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,[OffTikTokActivityDataList],,,,{'OffTikTokActivityDataList': 'object'}
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,"[App, BlockList]",,,,"{'App': 'number', 'BlockList': 'object'}"
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Settings,"[App, SettingsMap]",,,,"{'App': 'number', 'SettingsMap': {'Allow DownL..."
2,Comment,"{'Comments': {'App': 'number', 'CommentsList':...",Comment,Comments,"[App, CommentsList]",,,,"{'App': 'number', 'CommentsList': 'object'}"
3,Direct Message,{'Direct Messages': {'ChatHistory': {}}},Direct Message,Direct Messages,[ChatHistory],,,,{'ChatHistory': {}}
4,Income Plus Wallet Transactions,{'Transaction History': {'TransactionsList': '...,Income Plus Wallet Transactions,Transaction History,[TransactionsList],,,,{'TransactionsList': 'object'}
5,Location Review,{'Location Reviews': {'ReviewsList': 'object'}},Location Review,Location Reviews,[ReviewsList],,,,{'ReviewsList': 'object'}
6,Post,"{'Posts': {'VideoList': 'object'}, 'Recently D...",Post,Posts,[VideoList],,,,{'VideoList': 'object'}


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values,col_path_2_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,AdInterestCategories,string,,,{'AdInterestCategories': 'string'},string
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,ResponsesList,object,,,{'ResponsesList': 'object'},object
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,object,,,{'OffTikTokActivityDataList': 'object'},object
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,App,number,,,"{'App': 'number', 'BlockList': 'object'}",number
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,BlockList,object,,,"{'App': 'number', 'BlockList': 'object'}",object
...,...,...,...,...,...,...,...,...,...,...
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Purchases,BuyGifts,[BuyGifts],,,"{'SendGifts': {'SendGifts': 'object'}, 'BuyGif...",{'BuyGifts': 'object'}
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Searches,SearchList,"[Date, SearchTerm]",,,"{'SearchList': [{'Date': 'string', 'SearchTerm...","{'Date': 'string', 'SearchTerm': 'string'}"
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Share History,ShareHistoryList,object,,,{'ShareHistoryList': 'object'},object
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,"[Resolution, App Version, IDFA, GAID, Android ...",,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin..."


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values,col_path_2_values,col_path_3_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,AdInterestCategories,string,None,,{'AdInterestCategories': 'string'},string,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,ResponsesList,object,None,,{'ResponsesList': 'object'},object,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,object,None,,{'OffTikTokActivityDataList': 'object'},object,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,App,number,None,,"{'App': 'number', 'BlockList': 'object'}",number,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,BlockList,object,None,,"{'App': 'number', 'BlockList': 'object'}",object,None
...,...,...,...,...,...,...,...,...,...,...,...
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Android ID,string,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,IDFV,string,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Web ID,string,,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Watch History,VideoList,Date,string,,"{'VideoList': [{'Date': 'string', 'Link': 'str...","{'Date': 'string', 'Link': 'string'}",string


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,col_path_1_values,col_path_2_values,col_path_3_values,col_path_4_values
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Ad Interests,AdInterestCategories,string,None,None,{'AdInterestCategories': 'string'},string,None,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Instant Form Ads Responses,ResponsesList,object,None,None,{'ResponsesList': 'object'},object,None,None
0,Ads and data,{'Ad Interests': {'AdInterestCategories': 'str...,Ads and data,Off TikTok Activity,OffTikTokActivityDataList,object,None,None,{'OffTikTokActivityDataList': 'object'},object,None,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,App,number,None,None,"{'App': 'number', 'BlockList': 'object'}",number,None,None
1,App Settings,"{'Block List': {'App': 'number', 'BlockList': ...",App Settings,Block List,BlockList,object,None,None,"{'App': 'number', 'BlockList': 'object'}",object,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Android ID,string,None,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string,None
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,IDFV,string,None,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string,None
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Status,Status List,Web ID,string,None,"{'Status List': [{'Resolution': 'string', 'App...","{'Resolution': 'string', 'App Version': 'strin...",string,None
10,Your Activity,{'Activity Summary': {'ActivitySummaryMap': {'...,Your Activity,Watch History,VideoList,Date,string,None,"{'VideoList': [{'Date': 'string', 'Link': 'str...","{'Date': 'string', 'Link': 'string'}",string,None


In [ ]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values']
#, 'col_path_5_values'

merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest  #
    merged_df = merged_df.merge(df, on=common_columns, how="outer")



df_final = merged_df.replace('Missing', np.nan)


# Save the final merged DataFrame
df_final.to_csv(f"{main_path}TikTok/Final/Merged_structures_TT.csv", index=False)

/tmp/ipykernel_453491/3069983487.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = merged_df.replace('Missing', np.nan)
